# Using NLTK

Let's Check first that its only positive and Negative Reviews

In [48]:
import pandas as pd

file_path_new = '/content/dev.tsv'

new_data = pd.read_csv(file_path_new, sep='\t', header=None)

new_data.columns = ['review', 'label']

label_counts = new_data['label'].value_counts()

label_counts

,count
label,
1,444
0,428


In [49]:
import pandas as pd

file_path_new_train = '/content/train.tsv'

new_data_train = pd.read_csv(file_path_new_train, sep='\t', header=None)

new_data_train.columns = ['review', 'label']

label_counts_train = new_data_train['label'].value_counts()

label_counts_train

,count
label,
1,3610
0,3310


In [50]:
import pandas as pd

file_path_new_test = '/content/test.tsv'

new_data_test = pd.read_csv(file_path_new_test, sep='\t', header=None)

new_data_test.columns = ['review', 'label']

label_counts_test = new_data_test['label'].value_counts()

label_counts_test

,count
label,
0,912
1,909


Label the data to positive and negative reviews

In [69]:
import pandas as pd

file_path='/content/train.tsv'

data=pd.read_csv(file_path,sep='\t',header=None)

data.columns=['review','label']

label_mapping={0:'negative',1:'positive'}

data['label']=data['label'].map(label_mapping)

labeled_data=list(data.itertuples(index=False,name=None))

print(len(labeled_data))

6920


In [52]:
# Example: Checking for missing values
print(data.isnull().sum())

review    0
label     0
dtype: int64


In [53]:
!pip install nltk
!pip install scikit-learn

## Download necessary NLTK resources

In [54]:
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC


nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preprocessing the labeled data

In [55]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

preprocessed_data = []
labels = []

for review, label in labeled_data:
    tokens = word_tokenize(review.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_tweet = ' '.join(lemmatized_tokens)

    preprocessed_data.append(preprocessed_tweet)
    labels.append(label)

### Curiousity of me to see how the model will work with the data i gave him

In [56]:
# Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, labels, test_size=0.2, random_state=42)


In [57]:
# Vectorize the preprocessed data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)


In [58]:
# Train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_vectors, y_train)


SVC(kernel='linear')

In [59]:
# Evaluate the trained classifier on the testing set
y_pred = svm_classifier.predict(X_test_vectors)
classification_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report)

Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.75      0.78       671
    positive       0.78      0.84      0.81       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384



In [60]:
file_path = '/content/test.tsv'
data = pd.read_csv(file_path, sep='\t')

data.head()

unseen_reviews = data.iloc[:, 0]

unseen_reviews.head()


,"no movement , no yuks , not much of anything"
0,"a gob of drivel so sickly sweet , even the eag..."
1,"gangs of new york is an unapologetic mess , wh..."
2,"we never really feel involved with the story ,..."
3,this is one of polanski 's best films
4,take care of my cat offers a refreshingly diff...


In [61]:
import pandas as pd

file_path='/content/test.tsv'

data_test=pd.read_csv(file_path,sep='\t',header=None)

drop_columns=[1]

data_test.drop(columns=drop_columns,inplace=True)

data_test.columns=['review']

test_data=list(data.itertuples(index=False,name=None))

print(test_data[:5])

[("a gob of drivel so sickly sweet , even the eager consumers of moore 's pasteurized ditties will retch it up like rancid cr me br l e", 0), ('gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up', 0), ('we never really feel involved with the story , as all of its ideas remain just that abstract ideas', 0), ("this is one of polanski 's best films", 1), ('take care of my cat offers a refreshingly different slice of asian cinema', 1)]


In [63]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

preprocessed_test_data = []

for review in test_data:
    # Access the review text (first element of the tuple)
    text = review[0]
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_tweet = ' '.join(lemmatized_tokens)

    preprocessed_test_data.append(preprocessed_tweet)

In [67]:
preprocessed_test_data[:5]

["gob drivel sickly sweet , even eager consumer moore 's pasteurized ditty retch like rancid cr br l e",
 'gang new york unapologetic mess , whose saving grace end blowing everything',
 'never really feel involved story , idea remain abstract idea',
 "one polanski 's best film",
 'take care cat offer refreshingly different slice asian cinema']

In [64]:
analyzer = SentimentIntensityAnalyzer()

for review in preprocessed_test_data:
    sentiment_scores = analyzer.polarity_scores(review)
    print(f"Review: {review}")
    print(f"Sentiment Scores: {sentiment_scores}")
    print()

Streaming output truncated to the last 5000 lines.
Sentiment Scores: {'neg': 0.129, 'neu': 0.722, 'pos': 0.15, 'compound': 0.0987}

Review: film tedious impossible care whether boast true
Sentiment Scores: {'neg': 0.0, 'neu': 0.455, 'pos': 0.545, 'compound': 0.7184}

Review: regard , guard delivers
Sentiment Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Review: something true film buff enjoy
Sentiment Scores: {'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.7184}

Review: result solemn horrifying , yet strangely detached
Sentiment Scores: {'neg': 0.813, 'neu': 0.187, 'pos': 0.0, 'compound': -0.7717}

Review: gentle , compassionate drama grief healing
Sentiment Scores: {'neg': 0.283, 'neu': 0.177, 'pos': 0.54, 'compound': 0.4404}

Review: story uninteresting , song painfully undistinguished might giant ' one u may tuneless tune ever composed
Sentiment Scores: {'neg': 0.221, 'neu': 0.779, 'pos': 0.0, 'compound': -0.5267}

Review: 's cinematic equivalent good page t

In [72]:
import pandas as pd

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def classify_sentiment(score):
    if score > 0.5:
        return 'Positive'
    elif score < -0.5:
        return 'Negative'
    elif score<0.5 and score>-0.5:
        return 'Neutral'


# Create an empty dictionary to store the reviews, sentiment scores and the sentiment
reviews_data = {
    'Review': [],
    # 'Sentiment Scores':[],
    'Sentiment': []
}

# Loop through each review and analyze the sentiment
for review in preprocessed_test_data:
    sentiment_scores = analyzer.polarity_scores(review)

    # Classify the sentiment based on the compound score
    classified_sentiment = classify_sentiment(sentiment_scores['compound'])

    # Append the review, sentiment scores, and classified sentiment to the dictionary
    reviews_data['Review'].append(review)
    # reviews_data['Sentiment Scores'].append(sentiment_scores)
    reviews_data['Sentiment'].append(classified_sentiment)

# Convert the dictionary to a DataFrame
reviews_df = pd.DataFrame(reviews_data)

# Display the resulting DataFrame
reviews_df


,Review,Sentiment
0,"gob drivel sickly sweet , even eager consumer ...",Positive
1,"gang new york unapologetic mess , whose saving...",Neutral
2,"never really feel involved story , idea remain...",Neutral
3,one polanski 's best film,Positive
4,take care cat offer refreshingly different sli...,Neutral
...,...,...
1815,"often deadly boring , strange reading classic ...",Neutral
1816,"problem concept film concept poor one , 's sav...",Negative
1817,"safe conduct , however ambitious well intentio...",Positive
1818,"film made little wit , interest , professional...",Neutral


# Using Bert PreInstalled from HuggingFace
## I've done this incase we need the reviews to be from 1 to 5


In [73]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


# Load the pre-trained BERT model and tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [74]:
tokens=tokenizer.encode('this is amazing',return_tensors='pt')
result=model(tokens)
result.logits
int(torch.argmax(result.logits))+1

5

In [75]:
import numpy as np
import pandas as pd
file_path='/content/dev.tsv'

datanew=pd.read_csv(file_path,sep='\t',header=None)

datanew.drop(columns=[1],inplace=True)

dataf=pd.DataFrame(datanew)

dataf.columns=['review']

dataf

,review
0,one long string of cliches
1,if you 've ever entertained the notion of doin...
2,k 19 exploits our substantial collective fear ...
3,it 's played in the most straight faced fashio...
4,"there is a fabric of complex ideas here , and ..."
...,...
867,something like scrubbing the toilet
868,"smart , provocative and blisteringly funny"
869,"this one is definitely one to skip , even for ..."
870,charles ' entertaining film chronicles seinfel...


In [76]:
def sentiment_score(review):
  tokens=tokenizer.encode(review,return_tensors='pt')
  result=model(tokens)
  return int(torch.argmax(result.logits))+1

In [77]:
sentiment_score(dataf['review'].iloc[4])

5

In [78]:
dataf['Sentiment']=dataf['review'].apply(lambda x:sentiment_score(x[:512]))

In [79]:
dataf

,review,Sentiment
0,one long string of cliches,1
1,if you 've ever entertained the notion of doin...,1
2,k 19 exploits our substantial collective fear ...,1
3,it 's played in the most straight faced fashio...,4
4,"there is a fabric of complex ideas here , and ...",5
...,...,...
867,something like scrubbing the toilet,2
868,"smart , provocative and blisteringly funny",5
869,"this one is definitely one to skip , even for ...",1
870,charles ' entertaining film chronicles seinfel...,4


In [83]:
file_path='/content/train.tsv'

datanewests=pd.read_csv(file_path,sep='\t',header=None)

datanewests.drop(columns=[1],inplace=True)

trainframe=pd.DataFrame(datanewests)

trainframe.columns=['review']

trainframe

,review
0,"a stirring , funny and finally transporting re..."
1,apparently reassembled from the cutting room f...
2,they presume their audience wo n't sit still f...
3,this is a visually stunning rumination on love...
4,jonathan parker 's bartleby should have been t...
...,...
6915,"painful , horrifying and oppressively tragic ,..."
6916,take care is nicely performed by a quintet of ...
6917,"the script covers huge , heavy topics in a bla..."
6918,a seriously bad film with seriously warped log...


In [84]:
trainframe['Sentiment']=trainframe['review'].apply(lambda x:sentiment_score(x[:512]))

In [85]:
trainframe

,review,Sentiment
0,"a stirring , funny and finally transporting re...",5
1,apparently reassembled from the cutting room f...,1
2,they presume their audience wo n't sit still f...,4
3,this is a visually stunning rumination on love...,5
4,jonathan parker 's bartleby should have been t...,3
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,3
6917,"the script covers huge , heavy topics in a bla...",2
6918,a seriously bad film with seriously warped log...,1


In [86]:
file_path='/content/test.tsv'

datanewest=pd.read_csv(file_path,sep='\t',header=None)

datanewest.drop(columns=[1],inplace=True)

testframe=pd.DataFrame(datanewest)

testframe.columns=['review']

testframe

,review
0,"no movement , no yuks , not much of anything"
1,"a gob of drivel so sickly sweet , even the eag..."
2,"gangs of new york is an unapologetic mess , wh..."
3,"we never really feel involved with the story ,..."
4,this is one of polanski 's best films
...,...
1816,"an often deadly boring , strange reading of a ..."
1817,the problem with concept films is that if the ...
1818,"safe conduct , however ambitious and well inte..."
1819,"a film made with as little wit , interest , an..."


In [87]:
testframe['Sentiment']=testframe['review'].apply(lambda x:sentiment_score(x[:512]))

In [88]:
testframe

,review,Sentiment
0,"no movement , no yuks , not much of anything",1
1,"a gob of drivel so sickly sweet , even the eag...",1
2,"gangs of new york is an unapologetic mess , wh...",2
3,"we never really feel involved with the story ,...",2
4,this is one of polanski 's best films,5
...,...,...
1816,"an often deadly boring , strange reading of a ...",2
1817,the problem with concept films is that if the ...,2
1818,"safe conduct , however ambitious and well inte...",3
1819,"a film made with as little wit , interest , an...",5
